<a href="https://colab.research.google.com/github/tsarkov90/tf-style-transfer/blob/master/tf-style-transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural style transfer

Этот ноутбук использует глубокое обучение, чтобы на основе двух изображений составить другое. Этот алгоритм известен как *Neural Style Transfer*, и подробно изложен в работе <a href="https://arxiv.org/abs/1508.06576" class="external">A Neural Algorithm of Artistic Style</a>

Этот ноутбук демонстрирует оригинальный *NST*. Он оптимизирует содержание изображения в соответствии с определенным стилем.

*Neural Style Transfer* — это алгоритм, использующий два изображения: контентное изображение (*a content image*) и стилевое изображение (*a style image*), например, произведения искусства известного художника. Cмешивая их вместе, алгоритм выдает такую картинку, чтобы она выглядела как контентное изображение, но “окрашенное” в стилистике второго.

Вот пример работы алгоритма

Это у нас изображение контента.<br>
[Лабрадор куда-то смотрит](https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg)

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg" width="500px"/>

А это у нас стиль.<br>
[Композиция VII, Василий Кандинский](https://en.wikipedia.org/wiki/Composition_VII)

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg" width="500px"/>

Теперь, как бы это выглядело, если Кандинский решил написать картину этой собаки исключительно в таком стиле? Может быть что-то вроде этого?

<img src="https://tensorflow.org/tutorials/generative/images/stylized-image.png" style="width: 500px;"/>

## Установка


### Импорт и конфигурация модулей


In [0]:
import tensorflow as tf

In [0]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['axes.grid'] = False

In [0]:
import numpy as np
import PIL.Image
import functools
from tqdm.notebook import tqdm

In [0]:
def tensor_to_image(tensor):
  tensor = tensor * 255
  tensor = np.array(tensor, dtype=np.uint8)
    
  if np.ndim(tensor) > 3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  
  return PIL.Image.fromarray(tensor)

Скачиваем изображения и выбираем контент и стиль:

In [0]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

Можно не скачивать изображения и сразу назначить локальный путь

## Отрисовка изображения

Определим функцию, которая будет загружать изображение и ограничивать его размер до 512 пикселей.

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

Создадим функцию для показа изображения:

In [0]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [0]:
plt.figure(figsize=(12,12))

content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## Fast Style Transfer (TF-Hub)

Перед тем, как углубиться в детали оригинального алгоритма, давайте посмотрим, как модуль [TensorFlow Hub](https://tensorflow.org/hub) справляется с нашей задачей:

In [0]:
import tensorflow_hub as hub
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')

stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

## Представления контента и стиля

Будем использовать промежуточные слои модели для получения содержательных и стилевых представлений изображения. Начиная с входного слоя сети, первые несколько активаций слоев будут предсталять собой какие то простые шаблоны, по типу линии, точки, текстуры. Проходя по слоям через сеть, мы будем составлять все более и более сложные обьекты, по типу, колеса, деревья, окна, дома и так далее. 

В этом ноутбуке мы будем использовать готовую архитектуру сверточной нейронной сети **`VGG-19`**, которая будет предварително обучена для классификации изображений. 

Загрузим [VGG19](https://keras.io/applications/#vgg19) и протестируем её на нашей картинке:


In [0]:
x = tf.keras.applications.vgg19.preprocess_input(content_image * 255)
x = tf.image.resize(x, (224, 224))

vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

pred_proba = vgg(x)
pred_proba.shape

In [0]:
pred_top_5 = tf.keras.applications.vgg19.decode_predictions(pred_proba.numpy(), 5)[0]
imshow(content_image)
[(prob, class_name) for (number, class_name, prob) in pred_top_5]

Теперь загрузим **`VGG19`** без меток классов и выведем список названий слоёв

<img src='https://camo.githubusercontent.com/9ea2381f6f5172494f8c1a5f35c5012f5733a510/68747470733a2f2f692e696d6775722e636f6d2f314442593156302e706e67' width='500px'/>

In [0]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

for layer in vgg.layers:
    print(layer.name)

Выберем промежуточные слои из сети для представления стиля и содержания изображения:

In [0]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2']

# Style layer of interest
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

### Принцип работы алгоритма

Рассмотрим 1-й сверточный слой, который использует ядро `3x3` и обучает 64 карты признаков (*feature map*) для генерации изображения размерности `224х224х3`, принимая в качестве входных данных RGB-изображение.
Во время обучения эти карты признаков научились обнаруживать простые шаблоны, например, такие как линии, окружности или даже не имеющие никакого смысла для человеческого глаза шаблоны, которые тем не менее имеют огромное значение для этой модели. Такое «обнаружение» шаблонов называется обучением представления признаков.

Теперь давайте рассмотрим 10-й сверточный слой `VGG19`, который использует ядро `3x3` с 512 картами признаков для обучения и в итоге генерирует вывод представления изображения размерности `28x28x512`. Нейроны 10-го слоя уже могут обнаруживать более сложные шаблоны такие как, например, колесо автомобиля, окно или дерево и т.д.


Собственно вышеперечисленные свойства характерны для любой *CNN*, работа которой обычно интерпретируется как переход от конкретных особенностей изображения к более абстрактным деталям вплоть до выделения понятий высокого уровня. При этом сеть самонастраивается и вырабатывает необходимую иерархию абстрактных признаков (последовательности карт признаков), фильтруя маловажные детали и выделяя существенные.


Такая природа представления кодирования сама по себе является ключом к передаче стиля, который используется для вычисления функции потерь между сгенерированным изображением относительно изображения контента и изображения стиля. При обучении модели более десяти тысяч изображений на класс модель может генерировать аналогичное представление признаков для множества различных изображений, если они принадлежат к одному классу или имеют схожий контент или стиль.

## Строим модель

Нейронные сети в **`tf.keras.applications`** разработаны таким образом, чтобы вы могли извлекать значения промежуточного слоя с помощью **`Keras functional API`**.

Чтобы определить модель с помощью **`functional API`**, укажите входные и выходные данные:

`model = Model(inputs, outputs)`

Функция ниже строит модель **`VGG19`**, которая возвращает список выходов промежуточного слоя:

In [0]:
def vgg_layers(layer_names):
  ''' 
  Creates a vgg model that returns a list of intermediate output values.
  '''
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

Создаем модель, собственно:

In [0]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

#Look at the statistics of each layer's output
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

## Вычисляем стиль

Содержание изображения представлено значениями промежуточных карт объектов.

Оказывается, стиль изображения может быть описан средними и корреляциями между различными картами объектов. Вычислим матрицу Грама, содержащую эту информацию, взяв внешнее произведение вектора признаков с собой в каждом местоположении и усреднив это внешнее произведение по всем местоположениям. Эта матрица Гама может быть рассчитана для конкретного слоя следующим образом:

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

Реализуем это с помощью **`tf.linalg.einsum`**:

In [0]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

## Извлечение стиля и содержания

Строим модель, которая возвращает *тензор* контента и *тензор* стиля

In [0]:
class StyleContentModel(tf.keras.models.Model):

  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    '''
    Expects float input in [0,1]
    '''
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

При вызове изображения, эта модель возвращает матрицу Грама (стиль) для **`style_layers`** и контент для **`content_layers`**:

In [0]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

style_results = results['style']

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())


## Градиентный спуск

С помощью этого *извлекатора* стилей и контента теперь можно реализовать алгоритм передачи стилей. Сделаем это, рассчитав среднеквадратичную ошибку для вывода вашего изображения относительно каждой цели, а затем возьмите взвешенную сумму этих потерь.

Установим свой стиль и целевые значения контента:

In [0]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

Определим некую **`tf.Variable`**, которая будет содержать изображение для оптимизации. Чтобы сделать это быстро, инициализируем его с помощью изображения контента (**`tf.Variable`** должна иметь ту же размерность, что и изображение контента):

In [0]:
image = tf.Variable(content_image)

Поскольку это должно быть изображение, где значения пикселей это числа с плавающей точкой от 0 до 1, то определим функцию для *перевода* значений:

In [0]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

Создаём оптимизатор. Автор рекомендует `LBFGS`, но `Adam` тоже работает неплохо:

In [0]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

Чтобы оптимизировать это, используем взвешенную комбинацию двух потерь, чтобы получить общий убыток:

In [0]:
style_weight=1e-2
content_weight=1e4

In [0]:
def style_content_loss(outputs):
  style_outputs = outputs['style']
  content_outputs = outputs['content']
  style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                          for name in style_outputs.keys()])
  style_loss *= style_weight / num_style_layers

  content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                            for name in content_outputs.keys()])
  content_loss *= content_weight / num_content_layers
  
  loss = style_loss + content_loss
  return loss

Используем **`tf.GradientTape`** для обновления изображения:

In [0]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

А сейчас выполним несколько итераций:

In [0]:
train_step(image)
train_step(image)
train_step(image)
tensor_to_image(image)

У нас все работает, сделаем более продолжительную оптимизацию

In [0]:
%%time
epochs = 10
epoch_steps = 100

for n in range(epochs):
    print(f'Epoch: {n+1}')
    
    for m in tqdm(range(epoch_steps)):
        train_step(image)
    
    display.clear_output(wait=True)
    display.display(tensor_to_image(image))

## Total variation loss

Одним из недостатков этой базовой реализации является то, что она производит много высокочастотных артефактов. Уменьшим их, используя явный член регуляризации на высокочастотных компонентах изображения. В передаче стиля это часто называют полной потерей разброса/вариации (*total variation loss*):

In [0]:
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

In [0]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14,10))
plt.subplot(2,2,1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Original")

plt.subplot(2,2,2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2,2,3)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Styled")

plt.subplot(2,2,4)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Styled")

Как мы видим, увеличились высокочастотные компоненты (это шум).

Кроме того, с помощью этих шумов можно выделять обьекты на переднем и заднем плане. Такое может выполнять и [оператор Собеля](https://ru.wikipedia.org/wiki/Оператор_Собеля) (*Sobel Edge Detector*)

Применим его к контентному изображению:

In [0]:
plt.figure(figsize=(14,10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1,2,1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel-edges")
plt.subplot(1,2,2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel-edges")

Потеря регуляризации, связанная с этим, представляет собой сумму квадратов значений:

In [0]:
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

In [0]:
total_variation_loss(image).numpy()

Но нет никакой необходимости реализовывать его самостоятельно, TensorFlow включает в себя стандартную реализацию:

In [0]:
tf.image.total_variation(image).numpy()

## Перезапустим наш алгоритм

Выберем вес для **`total_variation_loss`**:

In [0]:
total_variation_weight = 30

А теперь учтём его в функции **`train_step`**:

In [0]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

Снова инициализируем оптимизируемую переменную:

In [0]:
image = tf.Variable(content_image)

И запустим алгоритм:

In [0]:
%%time
epochs = 10
epoch_steps = 100

for n in range(epochs):
    print(f'Epoch: {n+1}')
    
    for m in tqdm(range(epoch_steps)):
        train_step(image)
    
    display.clear_output(wait=True)
    display.display(tensor_to_image(image))

Сохраним результат

In [0]:
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)